# Análisis de sentimientos con Redes Neuronales

## Plan de acción

- Cargar el conjunto de datos de críticas de películas de IMDb (50.000 críticas).
- Preprocesar el conjunto de datos eliminando caracteres especiales, números, etc. de las reseñas de los usuarios + convertir las etiquetas de sentimiento positivo y negativo en números 1 y 0, respectivamente.
- Importar GloVe Word Embedding para crear un diccionario de incrustación + Utilizarlo para crear una matriz de incrustación.
- Entrenamiento del modelo usando Deep Learning en Keras para separar: Simple Neural Net, CNN y LSTM Models y analizar el rendimiento y los resultados del modelo.
- Por último, realizar predicciones sobre críticas de películas reales de IMDb.

In [1]:
# importando librerias
import pandas as pd
import numpy as np
from numpy import asarray, zeros, array
from numpy import zeros
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import Flatten, GlobalMaxPooling1D, Embedding, Conv1D, LSTM
from sklearn.model_selection import train_test_split

## Cargando el dataset

In [2]:
# cargando el dataset
df = pd.read_csv('IMDB_Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
# chequeando valores nulos
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [4]:
# chequeando las clases de la columna sentiment
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

## Procesamiento de datos

In [5]:
# visualizando una fila de la columna review para ver su estructura
df["review"][2]

'I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I\'d laughed at one of Woody\'s comedies in years (dare I say a decade?). While I\'ve never been impressed with Scarlet Johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than "Devil Wears Prada" and more interesting than "Superman" a great comedy to go see with friends.'

Se puede observar que nuestro texto contiene puntuaciones, paréntesis, etiquetas HTML y números, debemos procesar y limpiar el texto.

In [6]:
# usando expresiones regulares para eliminar las etiquetas HTML
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    '''Elimina etiquetas HTML: sustituye todo lo que haya entre la apertura y el cierre <> por espacio vacío'''
    return TAG_RE.sub('', text)

In [7]:
# descarga los conjuntos de palabras vacías (stopwords)
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alejandro\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
# usando expresiones regulares para limpiar el texto
def preprocess_text(sen):
    '''Limpia los datos de texto, dejando sólo 2 o más caracteres de longitud no-palabras compuestas de A-Z y a-z sólo
     en minúsculas'''
    
    # convierte el texto a minuscula
    sentence = sen.lower()

    # usa la funcion anterior para eliminar las etiquetas HTML
    sentence = remove_tags(sentence)

    # remueve puntuacion y numeros
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # eliminacion de un solo caracter
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)  # Cuando eliminamos el apóstrofo de la palabra "Mark's", el apóstrofo se sustituye por un espacio vacío. Por lo tanto, nos quedamos con un solo carácter "s" que estamos eliminando aquí.

    # remueve multiples espacios
    sentence = re.sub(r'\s+', ' ', sentence)  # A continuación, eliminamos todos los caracteres simples y lo reemplazamos por un espacio que crea múltiples espacios en nuestro texto. Por último, eliminamos también los espacios múltiples de nuestro texto.

    # removiendo Stopwords (and, the, etc)
    pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
    sentence = pattern.sub('', sentence)

    return sentence

In [9]:
# Llamada a la función preprocessing_text en movie_reviews
X = []
sentences = list(df['review'])
for sen in sentences:
    X.append(preprocess_text(sen))

In [10]:
# visualizando el texto anterior con las modificaciones
X[2]

'thought wonderful way spend time hot summer weekend sitting air conditioned theater watching light hearted comedy plot simplistic dialogue witty characters likable even well bread suspected serial killer may disappointed realize match point risk addiction thought proof woody allen still fully control style many us grown love laughed one woody comedies years dare say decade never impressed scarlet johanson managed tone sexy image jumped right average spirited young woman may crown jewel career wittier devil wears prada interesting superman great comedy go see friends '

In [11]:
# mapeando la columna sentiment donde positive es 1 y negative 0
y = df['sentiment']
y = np.array(list(map(lambda x: 1 if x=="positive" else 0, y)))

## Train Test Split

In [12]:
# separando datos de entrenamiento y datos de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

## Preparando Embedding Layer

Ahora escribamos el script para nuestra capa de incrustación(Embedding Layer). La capa de incrustación convierte nuestros datos textuales en forma numérica. A continuación, se utiliza como la primera capa para los modelos de deep learning en Keras.

In [13]:
# la capa de incrustación espera que las palabras estén en forma numérica 
# usando la función Tokenizer de la librería keras.preprocessing.text
# el método fit_on_text entrena el tokenizer 
# el método texts_to_sequences convierte las frases a su forma numérica
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(X_train)

X_train = word_tokenizer.texts_to_sequences(X_train)
X_test = word_tokenizer.texts_to_sequences(X_test)

In [14]:
# # añadir 1 para almacenar dimensiones de palabras para las que no existen incrustaciones de palabras preentrenadas
vocab_length = len(word_tokenizer.word_index) + 1
vocab_length

92394

In [15]:
# rellenando todas las revisiones a una longitud fija de 100
maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [16]:
# cargar GloVe word embeddings y crear un Embeddings Dictionary
embeddings_dictionary = dict()
glove_file = open('a2_glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [17]:
# creando Embedding Matrix que tenga 100 columnas
embedding_matrix = zeros((vocab_length, 100))
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [18]:
embedding_matrix.shape

(92394, 100)

## Entrenamiento del modelo

### Simple Neural Network

In [27]:
snn_model = Sequential()
embedding_layer = Embedding(vocab_length, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)

snn_model.add(embedding_layer)
snn_model.add(Flatten())
snn_model.add(Dense(1, activation='sigmoid'))

ValueError: Unrecognized keyword arguments passed to Embedding: {'weights': [array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.38251001,  0.14821   ,  0.60601002, ...,  0.058921  ,
         0.091112  ,  0.47283   ],
       [ 0.19915999, -0.049702  ,  0.24579   , ..., -0.068109  ,
         0.017651  ,  0.06455   ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.37771001,  0.22946   , -0.30311   , ..., -0.33610001,
         0.02637   , -0.72302002],
       [ 0.99882001,  0.044     ,  0.58508003, ..., -0.54869002,
         0.15141   ,  1.36880004]])], 'input_length': 100}